# 线性回归——从零开始

尽管强大的深度学习框架可以减少大量重复性工作，但若过于依赖它提供的便利，你就会很难深入理解深度学习是如何工作的。因此，我们的第一个教程是如何只利用ndarray和autograd来实现一个线性回归的训练。

## 线性回归

给定一个数据点集合`X`和对应的目标值`y`，线性模型的目标就是找到一条使用向量`w`和位移`b`描述的线，来尽可能地近似每个样本`X[i]`和`y[i]`。用数学符号来表示就是：

$$\boldsymbol{\hat{y}} = X \boldsymbol{w} + b$$

并最小化所有数据点上的平方误差

$$\sum_{i=1}^n (\hat{y}_i-y_i)^2.$$

你可能会对我们把古老的线性回归作为深度学习的一个样例表示奇怪。实际上线性模型是最简单、但也是最有用的神经网络。一个神经网络就是一个由节点（神经元）和有向边组成的集合。我们一般把一些节点组成层，每一层先从下面一层的节点获取输入，然后输出给上面的层使用。要计算一个节点值，我们需要将输入节点值做加权和（权数值即 `w`），然后再加上一个**激活函数（activation function）**。对于线性回归而言，它是一个两层神经网络，其中第一层是（下图橙色点）输入，每个节点对应输入数据点的一个维度，第二层是单输出节点（下图绿色点），它使用身份函数（$f(x)=x$）作为激活函数。

![](../img/onelayer.png)

## 创建数据集

这里我们使用一个数据集来尽量简单地解释清楚，真实的模型是什么样的。具体来说，我们使用如下方法来生成数据；随机数值 `X[i]`，其相应的标注为 `y[i]`：

`y[i] = 2 * X[i][0] - 3.4 * X[i][1] + 4.2 + noise`

使用数学符号表示：

$$y = X \cdot w + b + \eta, \quad \text{for } \eta \sim \mathcal{N}(0,\sigma^2)$$

这里噪音服从均值0和标准差为0.01的正态分布。

In [1]:
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet import gluon

data_root = '../data'
txt_root = data_root + '/CephalometricLandmark/AnnotationsByMD'
landmark_index = 15
num_examples = 150
num_inputs = 3
rate = 100
X = nd.zeros(shape=(num_examples, num_inputs))
y = nd.zeros(shape=(num_examples,))
for i in range(num_examples):
    txt_filename1 = txt_root + '/400_senior' + "/%03d.txt" % (i+1)
    with open(txt_filename1, 'r') as f:
        txts = f.read().split()
    x7 = int(txts[6].split(',')[0])
    X[i][0] = x7/rate
    
    x8 = int(txts[7].split(',')[0])
    X[i][1] = x8/rate
    
    x9 = int(txts[8].split(',')[0])
    X[i][2] = x9/rate
    
    x16 = int(txts[15].split(',')[0])
    y[i] = x16/rate
print(X)
print(y)


[[ 13.32999992  12.63000011  13.05000019]
 [ 13.81000042  13.19999981  13.63000011]
 [ 14.60000038  14.06000042  14.47999954]
 [ 14.68000031  14.18000031  14.48999977]
 [ 13.23999977  12.77999973  13.09000015]
 [ 13.88000011  13.44999981  13.68000031]
 [ 13.44999981  12.86999989  13.25      ]
 [ 13.67000008  13.18000031  13.53999996]
 [ 13.06999969  12.72000027  12.93999958]
 [ 13.65999985  13.31999969  13.59000015]
 [ 12.68999958  11.97999954  12.43999958]
 [ 16.20000076  15.81000042  16.04000092]
 [ 12.77999973  12.34000015  12.64000034]
 [ 12.44999981  11.97999954  12.28999996]
 [ 12.92000008  12.44999981  12.76000023]
 [ 14.35000038  13.93000031  14.21000004]
 [ 14.10999966  13.57999992  13.89000034]
 [ 12.48999977  11.93999958  12.26000023]
 [ 13.17000008  12.89000034  13.06999969]
 [ 13.13000011  12.52999973  12.94999981]
 [ 11.86999989  11.15999985  11.77999973]
 [ 14.34000015  13.89000034  14.23999977]
 [ 15.52999973  14.77999973  15.27999973]
 [ 13.31999969  12.71000004  13.1

注意到`X`的每一行是一个长度为2的向量，而`y`的每一行是一个长度为1的向量（标量）。

In [2]:
print(X[0], y[0])


[ 13.32999992  12.63000011  13.05000019]
<NDArray 3 @cpu(0)> 
[ 13.81999969]
<NDArray 1 @cpu(0)>


如果有兴趣，可以使用安装包中已包括的 Python 绘图包 `matplotlib`，生成第二个特征值 (`X[:, 1]`) 和目标值 `Y` 的散点图，更直观地观察两者间的关系。

## 数据读取

当我们开始训练神经网络的时候，我们需要不断读取数据块。这里我们定义一个函数它每次返回`batch_size`个随机的样本和对应的目标。我们通过python的`yield`来构造一个迭代器。

In [3]:
import random
batch_size = 15
def data_iter():
    # 产生一个随机索引
    idx = list(range(num_examples))
    random.shuffle(idx)
    for i in range(0, num_examples, batch_size):
        j = nd.array(idx[i:min(i+batch_size,num_examples)])
        yield nd.take(X, j), nd.take(y, j)

下面代码读取第一个随机数据块

In [4]:
for data, label in data_iter():
    print(data, label)
    break


[[ 13.60999966  13.06999969  13.43999958]
 [ 15.02999973  14.52999973  14.89999962]
 [ 12.61999989  12.02999973  12.36999989]
 [ 12.89000034  12.34000015  12.75      ]
 [ 12.92000008  12.44999981  12.76000023]
 [ 11.84000015  11.19999981  11.59000015]
 [ 15.19999981  14.68000031  15.05000019]
 [ 12.14999962  11.56999969  11.92000008]
 [ 13.          12.47000027  12.84000015]
 [ 13.27000046  12.47000027  13.02000046]
 [ 11.81000042  11.28999996  11.61999989]
 [ 13.47999954  13.06999969  13.32999992]
 [ 14.35000038  13.93000031  14.21000004]
 [ 12.90999985  12.39999962  12.77000046]
 [ 13.13000011  12.63000011  13.02000046]]
<NDArray 15x3 @cpu(0)> 
[ 14.32999992  15.85000038  13.14999962  13.60000038  13.65999985
  12.30000019  15.68999958  13.05000019  13.89000034  14.28999996
  12.55000019  14.43000031  14.94999981  13.44999981  13.90999985]
<NDArray 15 @cpu(0)>


## 初始化模型参数

下面我们随机初始化模型参数

In [5]:
w = nd.random_normal(shape=(num_inputs, 1))
b = nd.zeros((1,))
params = [w, b]
print(w.shape)
print(b.shape)

(3, 1)
(1,)


之后训练时我们需要对这些参数求导来更新它们的值，使损失尽量减小；因此我们需要创建它们的梯度。

In [6]:
for param in params:
    param.attach_grad()

## 定义模型

线性模型就是将输入和模型的权重（`w`）相乘，再加上偏移（`b`）：

In [7]:
def net(X):
    return nd.dot(X, w) + b

## 损失函数

我们使用常见的平方误差来衡量预测目标和真实目标之间的差距。

In [8]:
def square_loss(yhat, y):
    # 注意这里我们把y变形成yhat的形状来避免矩阵形状的自动转换
#     print(yhat.shape,y.shape)
    return nd.abs(yhat - y.reshape(yhat.shape))

## 优化

虽然线性回归有显式解，但绝大部分模型并没有。所以我们这里通过随机梯度下降来求解。每一步，我们将模型参数沿着梯度的反方向走特定距离，这个距离一般叫**学习率（learning rate）** `lr`。（我们会之后一直使用这个函数，我们将其保存在[utils.py](../utils.py)。）

In [9]:
def SGD(params, lr):
    for param in params:
#         print(param.grad)
        param[:] = param - lr * param.grad

## 训练

现在我们可以开始训练了。训练通常需要迭代数据数次，在这里使用`epochs`表示迭代总次数；一次迭代中，我们每次随机读取固定数个数据点，计算梯度并更新模型参数。

In [10]:
# 模型函数
def real_fn(X):
    return true_w[0] * X[:, 0] + true_w[1] * X[:, 1] + true_b
# 绘制损失随训练次数降低的折线图，以及预测值和真实值的散点图
def plot(losses, X, sample_size=100):
    xs = list(range(len(losses)))
    f, (fg1, fg2) = plt.subplots(1, 2)
    fg1.set_title('Loss during training')
    fg1.plot(xs, losses, '-r')
    fg2.set_title('Estimated vs real function')
    fg2.plot(X[:sample_size, 1].asnumpy(),
             net(X[:sample_size, :]).asnumpy(), 'or', label='Estimated')
    fg2.plot(X[:sample_size, 1].asnumpy(),
             real_fn(X[:sample_size, :]).asnumpy(), '*g', label='Real')
    fg2.legend()
    plt.show()

In [13]:
epochs = 50
learning_rate = .001
niter = 0
losses = []
moving_loss = 0
smoothing_constant = .01

# 训练
for e in range(epochs):    
    total_loss = 0

    for data, label in data_iter():
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
#         print(loss)
        loss.backward()
        SGD(params, learning_rate)
        total_loss += nd.sum(loss).asscalar()

        # 记录每读取一个数据点后，损失的移动平均值的变化；
        niter +=1
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss

        # correct the bias from the moving averages
        est_loss = moving_loss/(1-(1-smoothing_constant)**niter)
    print("Epoch %s, batch %s. Moving avg of loss: %s. Average loss: %f" % (e, niter, est_loss, total_loss/num_examples))

Epoch 0, batch 10. Moving avg of loss: 4.01661671144. Average loss: 4.007568
Epoch 1, batch 20. Moving avg of loss: 4.00612407663. Average loss: 3.989294
Epoch 2, batch 30. Moving avg of loss: 3.99487756756. Average loss: 3.968955
Epoch 3, batch 40. Moving avg of loss: 3.9849026891. Average loss: 3.953565
Epoch 4, batch 50. Moving avg of loss: 3.98205239113. Average loss: 3.964510
Epoch 5, batch 60. Moving avg of loss: 3.98177751503. Average loss: 3.974949
Epoch 6, batch 70. Moving avg of loss: 3.98301235055. Average loss: 3.980396
Epoch 7, batch 80. Moving avg of loss: 3.98525789958. Average loss: 3.983772
Epoch 8, batch 90. Moving avg of loss: 3.98651615434. Average loss: 3.980146
Epoch 9, batch 100. Moving avg of loss: 3.9855182539. Average loss: 3.970090
Epoch 10, batch 110. Moving avg of loss: 3.98468845267. Average loss: 3.966613
Epoch 11, batch 120. Moving avg of loss: 3.98744180037. Average loss: 3.991291
Epoch 12, batch 130. Moving avg of loss: 3.99032747547. Average loss: 3.9

训练完成后，我们可以比较学得的参数和真实参数

In [18]:
test_X = nd.zeros(shape=(150, num_inputs))
test_y = nd.zeros(shape=(150,))
for i in range(151,301):
    txt_filename1 = txt_root + '/400_senior' + "/%03d.txt" % i
    with open(txt_filename1, 'r') as f:
        txts = f.read().split()
    x7 = int(txts[6].split(',')[0])
    test_X[i-151][0] = x7/1000
    
    x8 = int(txts[7].split(',')[0])
    test_X[i-151][1] = x8/1000
    
    x9 = int(txts[8].split(',')[0])
    test_X[i-151][2] = x9/1000
    
    x16 = int(txts[15].split(',')[0])
    test_y[i-151] = x16/1000
py = net(X)
for i in range(150):
    print(rate*py[i],rate*y[i])




[ 1306.29125977]
<NDArray 1 @cpu(0)> 
[ 1382.]
<NDArray 1 @cpu(0)>

[ 1351.49914551]
<NDArray 1 @cpu(0)> 
[ 1459.]
<NDArray 1 @cpu(0)>

[ 1427.32739258]
<NDArray 1 @cpu(0)> 
[ 1526.]
<NDArray 1 @cpu(0)>

[ 1434.21875]
<NDArray 1 @cpu(0)> 
[ 1526.]
<NDArray 1 @cpu(0)>

[ 1292.82739258]
<NDArray 1 @cpu(0)> 
[ 1380.]
<NDArray 1 @cpu(0)>

[ 1354.64648438]
<NDArray 1 @cpu(0)> 
[ 1473.]
<NDArray 1 @cpu(0)>

[ 1315.69787598]
<NDArray 1 @cpu(0)> 
[ 1379.]
<NDArray 1 @cpu(0)>

[ 1335.46508789]
<NDArray 1 @cpu(0)> 
[ 1437.]
<NDArray 1 @cpu(0)>

[ 1274.01977539]
<NDArray 1 @cpu(0)> 
[ 1421.]
<NDArray 1 @cpu(0)>

[ 1331.53405762]
<NDArray 1 @cpu(0)> 
[ 1445.]
<NDArray 1 @cpu(0)>

[ 1244.03442383]
<NDArray 1 @cpu(0)> 
[ 1314.]
<NDArray 1 @cpu(0)>

[ 1580.484375]
<NDArray 1 @cpu(0)> 
[ 1693.]
<NDArray 1 @cpu(0)>

[ 1247.50964355]
<NDArray 1 @cpu(0)> 
[ 1356.]
<NDArray 1 @cpu(0)>

[ 1215.85791016]
<NDArray 1 @cpu(0)> 
[ 1312.]
<NDArray 1 @cpu(0)>

[ 1261.76147461]
<NDArray 1 @cpu(0)> 
[ 1366.]
<NDAr

## 小结

我们现在看到，仅仅是使用NDArray和autograd就可以很容易实现的一个模型。在接下来的教程里，我们会在此基础上，介绍更多现代神经网络的知识，以及怎样使用少量的MXNet代码实现各种复杂的模型。

## 练习

尝试用不同的学习率查看误差下降速度（收敛率）

## 讨论

欢迎扫码直达[本节内容讨论区](https://discuss.gluon.ai/t/topic/743)：